## Experiment parameters

In [ ]:
# 'save' or 'rerun'
save_or_rerun = 'save'

# 'passengers'
dataset = 'passengers'

## Model Parameters

In [ ]:
window_size = 12
train_size = int(12*4)

hermite_points = 2
montecarlo_runs = 10
variational_variance = 1e-6

In [ ]:
import datetime
from pathlib import Path
experiment_key = datetime.datetime.now().strftime("%Y%m%dT%H%M%S")
experiment_folder = Path('results') / dataset / experiment_key
print('experiment key:', experiment_key)

---

In [ ]:
import os
import datetime

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.FATAL)

import gpflow

import numpy as np
import pandas as pd

import sklearn as sk
import sklearn.preprocessing

import IPython.display as ipd
import matplotlib.pyplot as plt

from library.gplvm import GPLVM
from library.expectations import AnalyticExpectation, GaussHermiteExpectation, UnscentedExpectation, MonteCarloExpectation
import library.metrics
from library.helper import plot_process

from itertools import chain

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

np.random.seed(42)
tf.random.set_random_seed(42)

In [ ]:
def lag_dataframe(d,i):
    if i == 0:
        return d
    else:
        return d.shift(i).rename(lambda c: f'{c}_lag{i}', axis='columns')

def add_lag(lag,*data):
    return pd.concat([
        lag_dataframe(x,i) for x in data for i in range(0,lag)
    ], axis=1)

def kernel_name(k):
    if type(k) is gpflow.kernels.Sum:
        return '+'.join([kernel_name(k) for k in k.kernels])
    if type(k) is gpflow.kernels.Product:
        return '*'.join([kernel_name(k) for k in k.kernels])
    if type(k).__name__ == 'MLP':
        return f'MLP{k.layers}'
    else:
        return type(k).__name__

In [ ]:
dataset_path_friendly = dataset.replace(' ','_')
if save_or_rerun not in ['save','rerun']:
    raise Exception(f'Invalid operation {save_or_rerun}')
if dataset == 'passengers':
    y = pd.read_csv('data/international-airline-passengers.csv')
    y['Month'] = pd.to_datetime(y['Month'])
    y = y.sort_values('Month').set_index('Month')
    y['passengers in thousands'] = y['passengers in thousands'].astype(float)
    t = y.index
else:
    raise Exception(f'Unknown Dataset {dataset}')

In [ ]:
def windowfy(size, data):
    windowfied_data = pd.concat({0:data}, axis=1, names=['lag', *data.columns.names])
    for i in range(1, size):
        windowfied_data = pd.concat([windowfied_data, pd.concat({i:data.shift(i)}, axis=1)], axis=1)
    return windowfied_data

In [ ]:
X = windowfy(window_size,y)
X_train = X.iloc[window_size-1:window_size+train_size-1].values

X_train_var = variational_variance*np.ones(X_train.shape)
y_train = y.iloc[window_size:window_size+train_size].values

y_scaler = sk.preprocessing.StandardScaler()
y_train = y_scaler.fit_transform(y_train)
X_train = (X_train - y_scaler.mean_)/y_scaler.scale_

inducing_points = X_train

In [ ]:
kernels = [
    gpflow.kernels.RBF(window_size, ARD=True) + gpflow.kernels.Linear(window_size, ARD=True),
    gpflow.kernels.Periodic(window_size) + gpflow.kernels.RBF(window_size, ARD=True) + gpflow.kernels.Linear(window_size, ARD=True),
]
expectations_and_runs = [
    (AnalyticExpectation(), 1),
    (UnscentedExpectation(), 1),
    (GaussHermiteExpectation(hermite_points, din=window_size), 1),
    (MonteCarloExpectation(hermite_points**window_size), montecarlo_runs),
    (MonteCarloExpectation(200), montecarlo_runs),
    (MonteCarloExpectation(hermite_points*window_size), montecarlo_runs)
]

def make_model_params(k, expectation=None, predict_mode='GP'):
    return {
        'expectation': expectation, 'train_mode': 'GP', 'predict_mode': predict_mode,
        'X_mean': X_train.copy(), 'X_var': X_train_var.copy(), 'Y': y_train,
        'kern': k, 'Z': inducing_points.copy(), 'M': inducing_points.shape[0]
    }

def init_kernel_hypers(k):
    if type(k) is gpflow.kernels.Sum:
        for subk in k.kernels:
            init_kernel_hypers(subk)
    elif isinstance(k, gpflow.kernels.Stationary):
        k.lengthscales = 10*np.sqrt(np.var(X_train, axis=0))
        k.variance = np.var(y_train)
    elif isinstance(k, gpflow.kernels.Static):
        pass
    elif type(k) is gpflow.kernels.Linear:
        k.variance = (1/X_train.shape[1])*1e-2*np.var(y_train)*np.ones(window_size)
    elif type(k) is gpflow.kernels.Periodic:
        k.variance = np.var(y_train)
    elif type(k).__name__ == 'MLP':
        k.rbf.variance = np.var(y_train)
    else:
        raise Exception(f"Can't handle {type(k)}")

gp_narxs = pd.Series({kernel_name(k): GPLVM(**make_model_params(k)) for k in kernels})
for m in gp_narxs:
    m.likelihood.variance = 0.01 * np.var(y_train)
    m.X_mean.trainable = False
    m.X_var.trainable = False
    m.feature.trainable = False
    init_kernel_hypers(m.kern)

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)
for m in gp_narxs:
    ipd.display(ipd.HTML(f'<h4>{kernel_name(m.kern)}</h4>'))
    print('Start',datetime.datetime.now().strftime("%I:%M %p"))
    %time gpflow.training.ScipyOptimizer().minimize(m, maxiter=5000)
tf.logging.set_verbosity(tf.logging.FATAL)

for m in gp_narxs:
    m.X_var.value[:,:] = m.likelihood.variance.value
print('Actual finish', datetime.datetime.now().strftime("%I:%M %p"))

In [ ]:
gplvms = pd.Series({
    ('GPLVM', e.__name__, kernel_name(k), i): GPLVM(**make_model_params(k,e,predict_mode='GPLVM'))
    for e,r in expectations_and_runs
    for k in kernels
    for i in range(r)
    if not (e.__name__ == 'analytic' and kernel_name(k) != 'SquaredExponential+Linear')
})

for gplvm in gplvms:
    gplvm.assign(gp_narxs[kernel_name(gplvm.kern)].read_trainables())

In [ ]:
models_index = pd.MultiIndex.from_tuples(
    [
        *[('NARX', 'analytic', kernel_name(k), 0) for k in kernels],
        *gplvms.index
    ],
    names=['mode','expectation', 'kernel','run']
)

result_columns = pd.MultiIndex.from_tuples(
    [(*x, r) for x in models_index for r in ['mean', 'variance']],
    names=[*models_index.names, None]
)

In [ ]:
def free_simulate(m, propagate_uncertainty):
    starting_loc = X.index[window_size-1]

    means = pd.DataFrame(index=X.index, columns=X.columns, dtype=float).iloc[window_size-1:]
    means.loc[starting_loc] = (X.loc[starting_loc] - y_scaler.mean_)/y_scaler.scale_

    variances = pd.DataFrame(index=X.index, columns=X.columns, dtype=float).iloc[window_size-1:]
    variances.loc[starting_loc] = m.likelihood.variance.value


    for t_curr, t_next in zip(means.index, means.index[1:]):
        if not propagate_uncertainty:
            next_mean, next_variance = m.predict_y(means.loc[[t_curr]])
        else:
            next_mean, next_variance = m.predict_y_uncertain(means.loc[[t_curr]], variances.loc[[t_curr]])

        means.loc[t_next,0] = next_mean.squeeze()
        means.loc[t_next,pd.IndexSlice[1:]] = means.loc[t_curr,pd.IndexSlice[0:window_size-2]].values

        variances.loc[t_next,0] = next_variance.squeeze()
        variances.loc[t_next,pd.IndexSlice[1:]] = variances.loc[t_curr,pd.IndexSlice[0:window_size-2]].values
        
    return pd.concat({'mean':means[0], 'variance': variances[0]}, axis=1).swaplevel(axis=1).iloc[1:]

In [ ]:
narx_results = (free_simulate(m, propagate_uncertainty=False) for m in gp_narxs)
gplvm_results = (free_simulate(m, propagate_uncertainty=True) for m in gplvms)

In [ ]:
results = pd.concat(chain(narx_results, gplvm_results), axis=1)
results.columns = result_columns
results.loc[:, pd.IndexSlice[:,:,:,:,'mean']] = y_scaler.inverse_transform(results.loc[:, pd.IndexSlice[:,:,:,:,'mean']])
results.loc[:, pd.IndexSlice[:,:,:,:,'variance']] = y_scaler.var_[0] * results.loc[:, pd.IndexSlice[:,:,:,:,'variance']]
if save_or_rerun == 'save':
    experiment_folder.mkdir(parents=True)
    results.to_hdf(experiment_folder/'points.hdf', key='points')

In [ ]:
metrics = pd.DataFrame({
    'RMSE': [
        np.sqrt(sk.metrics.mean_squared_error(y.iloc[window_size+train_size:], results.iloc[train_size:][(*k, 'mean')]))
        if results.iloc[train_size+1:][(*k, 'mean')].isnull().any() != True
        else np.nan
        for k in models_index
    ],
    'NLPD': [
        library.metrics.negative_log_predictive_density(
            y.iloc[window_size+train_size:].values.reshape(-1,1),
            results.iloc[train_size:][[(*k, 'mean')]].values,
            results.iloc[train_size:][[(*k, 'variance')]].values,
        )[0]
        for k in models_index
    ]
}, index=models_index).sort_index(axis=1)

if save_or_rerun == 'save':
    metrics['NLPD'].unstack('run').T.describe().loc[['mean','std']].T.to_csv(experiment_folder/f'nlpd_mean.csv')
    metrics['RMSE'].unstack('run').T.describe().loc[['mean','std']].T.to_csv(experiment_folder/f'rmse_mean.csv')
    metrics.to_csv(experiment_folder/f'acc.csv')

In [ ]:
if save_or_rerun in ['save', 'loadsave']:
    os.makedirs(f"./results/figs/{dataset}/", exist_ok=True)
for (mode,e,k,run), kMetrics in metrics.sort_values(by='NLPD', kind='mergesort').iterrows():
    figname = f'{k}_{run}'
    print(figname, *kMetrics.items())
    f, ax = plt.subplots(1,1, figsize=np.array([5,2])*1.5)
    ax.plot(t,y.values.reshape(-1,1), label='Data', linestyle='-')
    a = ax.get_ylim()
    plot_process(ax,t.values[window_size:],results[mode,e,k,run], color='C1', label='Prediction')
    # plot_process(ax,t.values[window_size:],[
    #   results[mode,e,k,run]['mean'].values.reshape(-1,1), results[mode,e,k,run]['variance'].values.reshape(-1,1)
    # ], color='C1', label='Prediction')
    ax.set_ylim(a)
    ax.legend()
    if save_or_rerun in ['save', 'loadsave']:
        plot_folders = experiment_folder/mode/e
        plot_folders.mkdir(parents=True, exist_ok=True)
        plt.savefig(plot_folders/f'{figname}.pdf', bbox_inches='tight')
    else:
        plt.show()
    plt.close(f)